In [123]:
import pandas as pd
import plotly.express as px
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf

In [81]:

df = pd.read_csv("Data/Data.csv")
df = df[["country", "commodity", "year", "loss_percentage", "activity", "food_supply_stage", "cause_of_loss"]]
df

,country,commodity,year,loss_percentage,activity,food_supply_stage,cause_of_loss
0,Myanmar,"Groundnuts, excluding shelled",2009,5.22,NaN,Whole supply chain,NaN
1,Myanmar,"Groundnuts, excluding shelled",2008,5.43,NaN,Whole supply chain,NaN
2,Myanmar,"Groundnuts, excluding shelled",2007,5.61,NaN,Whole supply chain,NaN
3,Myanmar,"Groundnuts, excluding shelled",2006,5.40,NaN,Whole supply chain,NaN
4,Myanmar,"Groundnuts, excluding shelled",2005,5.00,NaN,Whole supply chain,NaN
...,...,...,...,...,...,...,...
27768,Zambia,Millet,2000,1.27,Storage,Farm,NaN
27769,Zambia,Millet,2000,2.50,Winnowing,Farm,NaN
27770,Zambia,Millet,2000,1.00,Transportation,Transport,NaN
27771,Zambia,Millet,2000,2.38,Storage,Storage,NaN


In [82]:
def focus_on(country, commodity):
    new_df=df[df.country==country]
    new_df=new_df[new_df.commodity==commodity]
    return new_df

new_df=focus_on("Kenya","Maize (corn)")


new_df=focus_on("Benin","Maize (corn)")
new_df

,country,commodity,year,loss_percentage,activity,food_supply_stage,cause_of_loss
2448,Benin,Maize (corn),2020,2.65,Storage,Storage,NaN
2449,Benin,Maize (corn),2020,3.04,Storage,Farm,NaN
2450,Benin,Maize (corn),2020,6.42,"Drying, Harvesting",Harvest,NaN
2451,Benin,Maize (corn),2020,4.00,Drying,Farm,NaN
2452,Benin,Maize (corn),2020,2.36,Transportation,Farm,NaN
...,...,...,...,...,...,...,...
3068,Benin,Maize (corn),2000,1.65,Transportation,Transport,NaN
3069,Benin,Maize (corn),2000,3.05,Storage,Farm,NaN
3070,Benin,Maize (corn),2000,4.00,Drying,Farm,NaN
3071,Benin,Maize (corn),2000,2.36,Transportation,Farm,NaN


In [83]:
new_df=new_df.reset_index()

In [84]:
new_df.sort_values(by=(["index"]),ascending=False, inplace=True)

In [85]:
train=new_df[:-round(len(new_df)*0.2)]
test=new_df[-round(len(new_df)*0.2):]

In [86]:
train

,index,country,commodity,year,loss_percentage,activity,food_supply_stage,cause_of_loss
151,3072,Benin,Maize (corn),2000,6.42,"Drying, Harvesting",Harvest,NaN
150,3071,Benin,Maize (corn),2000,2.36,Transportation,Farm,NaN
149,3070,Benin,Maize (corn),2000,4.00,Drying,Farm,NaN
148,3069,Benin,Maize (corn),2000,3.05,Storage,Farm,NaN
147,3068,Benin,Maize (corn),2000,1.65,Transportation,Transport,NaN
...,...,...,...,...,...,...,...,...
34,2566,Benin,Maize (corn),2016,2.36,Transportation,Farm,NaN
33,2565,Benin,Maize (corn),2016,3.00,Storage,Farm,NaN
32,2564,Benin,Maize (corn),2016,4.00,Drying,Farm,NaN
31,2563,Benin,Maize (corn),2016,2.65,Storage,Storage,NaN


In [87]:
test

,index,country,commodity,year,loss_percentage,activity,food_supply_stage,cause_of_loss
29,2561,Benin,Maize (corn),2016,1.65,Transportation,Transport,NaN
28,2560,Benin,Maize (corn),2016,6.42,"Drying, Harvesting",Harvest,NaN
27,2538,Benin,Maize (corn),2017,1.65,Transportation,Transport,NaN
26,2537,Benin,Maize (corn),2017,6.42,"Drying, Harvesting",Harvest,NaN
25,2536,Benin,Maize (corn),2017,2.65,Storage,Storage,NaN
24,2535,Benin,Maize (corn),2017,4.00,Drying,Farm,NaN
23,2534,Benin,Maize (corn),2017,1.32,"Shelling, Threshing",Farm,NaN
22,2533,Benin,Maize (corn),2017,2.36,Transportation,Farm,NaN
21,2532,Benin,Maize (corn),2017,3.04,Storage,Farm,NaN
20,2510,Benin,Maize (corn),2018,2.36,Transportation,Farm,NaN


In [127]:
train

,index,country,commodity,year,loss_percentage,activity,food_supply_stage,cause_of_loss
151,3072,Benin,Maize (corn),2000,0.198598,"Drying, Harvesting",Harvest,NaN
150,3071,Benin,Maize (corn),2000,0.040498,Transportation,Farm,NaN
149,3070,Benin,Maize (corn),2000,0.104361,Drying,Farm,NaN
148,3069,Benin,Maize (corn),2000,0.067368,Storage,Farm,NaN
147,3068,Benin,Maize (corn),2000,0.012850,Transportation,Transport,NaN
...,...,...,...,...,...,...,...,...
34,2566,Benin,Maize (corn),2016,0.040498,Transportation,Farm,NaN
33,2565,Benin,Maize (corn),2016,0.065421,Storage,Farm,NaN
32,2564,Benin,Maize (corn),2016,0.104361,Drying,Farm,NaN
31,2563,Benin,Maize (corn),2016,0.051791,Storage,Storage,NaN


In [130]:
scaler = MinMaxScaler(feature_range=(0,1))
train["loss_percentage"] = scaler.fit_transform(train[["loss_percentage"]].to_numpy())
test["loss_percentage"]=scaler.transform(test[["loss_percentage"]].to_numpy())






/var/folders/t6/nkv95bj97gv9j73w9bfk2p0r0000gn/T/ipykernel_14676/1809770144.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["loss_percentage"] = scaler.fit_transform(train[["loss_percentage"]].to_numpy())
/var/folders/t6/nkv95bj97gv9j73w9bfk2p0r0000gn/T/ipykernel_14676/1809770144.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["loss_percentage"]=scaler.transform(test[["loss_percentage"]].to_numpy())


In [132]:
Y_data

151    0.198598
150    0.040498
149    0.104361
148    0.067368
147    0.012850
         ...   
34     0.040498
33     0.065421
32     0.104361
31     0.051791
30     0.000000
Name: loss_percentage, Length: 122, dtype: float64

In [98]:
test_arr=test.to_numpy()

In [102]:
 def custom_ts_multi_data_prep(dataset, target, start, end, window, horizon):
     X = []
     y = []
     start = start + window
     if end is None:
         end = len(dataset) - horizon
     for i in range(start, end):
         indices = range(i-window, i)
         X.append(dataset[indices])
         indicey = range(i+1, i+1+horizon)
         y.append(target[indicey])
     return np.array(X), np.array(y)

hist_window = 8
horizon = 1
TRAIN_SPLIT = 30
x_train, y_train = custom_ts_multi_data_prep(X_data, Y_data, 0, TRAIN_SPLIT, hist_window, horizon)
x_vali, y_vali = custom_ts_multi_data_prep(X_data, Y_data, TRAIN_SPLIT, None, hist_window, horizon

In [103]:
testX

array([[[2561, 'Benin', 'Maize (corn)', 2016, 0.01285046728971962,
         'Transportation', 'Transport', nan],
        [2560, 'Benin', 'Maize (corn)', 2016, 0.19859813084112146,
         'Drying, Harvesting', 'Harvest', nan],
        [2538, 'Benin', 'Maize (corn)', 2017, 0.01285046728971962,
         'Transportation', 'Transport', nan]],

       [[2560, 'Benin', 'Maize (corn)', 2016, 0.19859813084112146,
         'Drying, Harvesting', 'Harvest', nan],
        [2538, 'Benin', 'Maize (corn)', 2017, 0.01285046728971962,
         'Transportation', 'Transport', nan],
        [2537, 'Benin', 'Maize (corn)', 2017, 0.19859813084112146,
         'Drying, Harvesting', 'Harvest', nan]],

       [[2538, 'Benin', 'Maize (corn)', 2017, 0.01285046728971962,
         'Transportation', 'Transport', nan],
        [2537, 'Benin', 'Maize (corn)', 2017, 0.19859813084112146,
         'Drying, Harvesting', 'Harvest', nan],
        [2536, 'Benin', 'Maize (corn)', 2017, 0.05179127725856696,
         'Storage

In [104]:
print("trainX Shape-- ",trainX.shape)
print("trainY Shape-- ",trainY.shape)


print("testX Shape-- ",testX.shape)
print("testY Shape-- ",testY.shape)



trainX Shape--  (119, 3, 8)
trainY Shape--  (119,)
testX Shape--  (27, 3, 8)
testY Shape--  (27,)


In [105]:
trainX[0]

array([[3072, 'Benin', 'Maize (corn)', 2000, 0.19859813084112146,
        'Drying, Harvesting', 'Harvest', nan],
       [3071, 'Benin', 'Maize (corn)', 2000, 0.04049844236760123,
        'Transportation', 'Farm', nan],
       [3070, 'Benin', 'Maize (corn)', 2000, 0.10436137071651089,
        'Drying', 'Farm', nan]], dtype=object)

In [106]:
trainY[0]

0.06736760124610591

In [107]:
train

,index,country,commodity,year,loss_percentage,activity,food_supply_stage,cause_of_loss
151,3072,Benin,Maize (corn),2000,0.198598,"Drying, Harvesting",Harvest,NaN
150,3071,Benin,Maize (corn),2000,0.040498,Transportation,Farm,NaN
149,3070,Benin,Maize (corn),2000,0.104361,Drying,Farm,NaN
148,3069,Benin,Maize (corn),2000,0.067368,Storage,Farm,NaN
147,3068,Benin,Maize (corn),2000,0.012850,Transportation,Transport,NaN
...,...,...,...,...,...,...,...,...
34,2566,Benin,Maize (corn),2016,0.040498,Transportation,Farm,NaN
33,2565,Benin,Maize (corn),2016,0.065421,Storage,Farm,NaN
32,2564,Benin,Maize (corn),2016,0.104361,Drying,Farm,NaN
31,2563,Benin,Maize (corn),2016,0.051791,Storage,Storage,NaN


In [121]:
def build_model(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(50,return_sequences=True,input_shape=(30,5)))
    grid_model.add(LSTM(50))
    grid_model.add(Dropout(0.2))
    grid_model.add(Dense(1))
    grid_model.compile(loss = 'mse',optimizer = optimizer)
    return grid_model


grid_model = KerasRegressor(build_fn=build_model, verbose=1, validation_data=(testX,testY))


parameters = {'batch_size' : [16,20],
              'epochs' : [8,10],
              'optimizer' : ['adam','Adadelta'] }

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            cv = 2)

/var/folders/t6/nkv95bj97gv9j73w9bfk2p0r0000gn/T/ipykernel_14676/854486651.py:11: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_model = KerasRegressor(build_fn=build_model, verbose=1, validation_data=(testX,testY))


In [122]:
grid_search = grid_search.fit(trainX,trainY)

2022-07-05 15:20:44.905656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: 
All the 16 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
16 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/murathansaygili/Desktop/SIC/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/murathansaygili/Desktop/SIC/venv/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 164, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/Users/murathansaygili/Desktop/SIC/venv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/Users/murathansaygili/Desktop/SIC/venv/lib/python3.9/site-packages/tensorflow/python/

In [ ]:
batch_size = 256
buffer_size = 150
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(buffer_size).batch(batch_size).repeat()
val_data = tf.data.Dataset.from_tensor_slices((x_vali, y_vali))
val_data = val_data.batch(batch_size).repeat()